# Client

Demo of a client interacting with a remote agent that can use history.

See relevant documentation about agents:

* Creating a custom agent: https://python.langchain.com/docs/modules/agents/how_to/custom_agent
* Streaming with agents: https://python.langchain.com/docs/modules/agents/how_to/streaming#custom-streaming-with-events
* General streaming documentation: https://python.langchain.com/docs/expression_language/streaming

You can interact with this via API directly

In [1]:
import requests

inputs = {"input": {"input": "what is the length of the word audioee?", "chat_history": []}}
response = requests.post("http://localhost:8000/invoke", json=inputs)

response.json()

{'output': {'output': 'The length of the word "audioee" is 7.'},
 'callback_events': [],
 'metadata': {'run_id': '1847be77-f53c-40ba-b88d-06af3a598b6e'}}

You can also interact with this via the RemoteRunnable interface (to use in other chains)

In [2]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/")

Remote runnable has the same interface as local runnables

In [3]:
from langchain_core.messages import HumanMessage, AIMessage

In [4]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print('AI: Bye bye human')
        break
    ai = await remote_runnable.ainvoke({"input": human, "chat_history": chat_history})
    print(f"AI: {ai['output']}")
    chat_history.extend([HumanMessage(content=human), AIMessage(content=ai['output'])])

Human (Q/q to quit):  hello


AI: Hello! How can I assist you today?


Human (Q/q to quit):  my name is eugene


AI: Nice to meet you, Eugene! How can I help you today?


Human (Q/q to quit):  what is my name


AI: Your name is Eugene.


Human (Q/q to quit):  what is the length of my name


AI: The length of your name, Eugene, is 6 characters.


Human (Q/q to quit):  q


AI: Bye bye human


## Stream

Please note that streaming alternates between actions and observations. It does not stream individual tokens!

To stream individual tokens, we need to use the astream events endpoint (see below).

In [5]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print('AI: Bye bye human')
        break

    ai = None
    print("AI: ")
    async for chunk in remote_runnable.astream({"input": human, "chat_history": chat_history}):
        # Agent Action
        if "actions" in chunk:
            for action in chunk["actions"]:
                print(
                    f"Calling Tool ```{action['tool']}``` with input ```{action['tool_input']}```"
                )
        # Observation
        elif "steps" in chunk:
            for step in chunk["steps"]:
                print(f"Got result: ```{step['observation']}```")
        # Final result
        elif "output" in chunk:
            print(chunk['output'])
            ai = AIMessage(content=chunk['output'])
        else:
            raise ValueError
        print("------")        
    chat_history.extend([HumanMessage(content=human), ai])

Human (Q/q to quit):  hello


AI: 
Hello! How can I assist you today?
------


Human (Q/q to quit):  my name is eugene


AI: 
Nice to meet you, Eugene! How can I help you today?
------


Human (Q/q to quit):  what is my name


AI: 
Your name is Eugene.
------


Human (Q/q to quit):  what's the length of my name?


AI: 
Calling Tool ```word_length``` with input ```{'word': 'Eugene'}```
------
Got result: ```6```
------
The length of your name, Eugene, is 6 characters.
------


Human (Q/q to quit):  q


AI: Bye bye human


## Stream Events

In [4]:
chat_history = []

while True:
    human = input("Human (Q/q to quit): ")
    if human in {"q", "Q"}:
        print('AI: Bye bye human')
        break
    ai = None
    print("AI: ")
    async for event in remote_runnable.astream_events(
        {"input": human, "chat_history": chat_history},
        version="v1",
    ):
        kind = event["event"]
        if kind == "on_chain_start":
            if (
                event["name"] == "agent"
            ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
                print(
                    f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
                )
        elif kind == "on_chain_end":
            if (
                event["name"] == "agent"
            ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
                print()
                print("--")
                print(
                    f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
                )
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                # Empty content in the context of OpenAI means
                # that the model is asking for a tool to be invoked.
                # So we only print non-empty content
                print(content, end="|")
        elif kind == "on_tool_start":
            print("--")
            print(
                f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
            )
        elif kind == "on_tool_end":
            print(f"Done tool: {event['name']}")
            print(f"Tool output was: {event['data'].get('output')}")
            print("--")    


Human (Q/q to quit):  hello! my name is eugene


AI: 
Starting agent: agent with input: {'input': 'hello! my name is eugene', 'chat_history': []}
Hello| Eugene|!| How| can| I| assist| you| today|?|
--
Done agent: agent with output: Hello Eugene! How can I assist you today?


Human (Q/q to quit):  what's the length of my name?


AI: 
Starting agent: agent with input: {'input': "what's the length of my name?", 'chat_history': []}
--
Starting tool: word_length with inputs: {'word': 'my name'}
Done tool: word_length
Tool output was: 7
--
The| length| of| your| name|,| "|my| name|",| is| |7| characters|.|
--
Done agent: agent with output: The length of your name, "my name", is 7 characters.


Human (Q/q to quit):  could you tell me a long story about the length of my name?


AI: 
Starting agent: agent with input: {'input': 'could you tell me a long story about the length of my name?', 'chat_history': []}
Once| upon| a| time|,| there| was| a| person| named| [|Your| Name|].| Now|,| [|Your| Name|]| had| a| very| unique| and| special| name|.| It| was| a| name| that| carried| a| lot| of| meaning| and| significance|.| People| often| wondered| about| the| length| of| [|Your| Name|]'|s| name| and| how| it| compared| to| others|.

|One| day|,| [|Your| Name|]| decided| to| embark| on| a| journey| to| discover| the| true| length| of| their| name|.| They| traveled| far| and| wide|,| seeking| the| wisdom| of| s|ages| and| scholars| who| were| known| for| their| knowledge| of| names| and| their| lengths|.

|Along| the| way|,| [|Your| Name|]| encountered| many| interesting| characters| who| had| their| own| stories| to| tell| about| the| lengths| of| their| names|.| Some| had| short| names| that| were| easy| to| remember|,| while| others| had| long| names| that| seemed| 

Human (Q/q to quit):  that's not what I wanted. My name is eugene. calculate the length of my name and tell me a story about the result.


AI: 
Starting agent: agent with input: {'input': "that's not what I wanted. My name is eugene. calculate the length of my name and tell me a story about the result.", 'chat_history': []}
--
Starting tool: word_length with inputs: {'word': 'eugene'}
Done tool: word_length
Tool output was: 6
--
The| length| of| your| name|,| Eugene|,| is| |6| characters|.| Now|,| let| me| tell| you| a| story| about| the| result|.

|Once| upon| a| time|,| in| a| land| far| away|,| there| was| a| young| boy| named| Eugene|.| He| had| a| special| power| -| the| power| to| bring| joy| and| laughter| to| everyone| he| met|.| Eugene|'s| name|,| with| its| |6| letters|,| perfectly| reflected| his| vibrant| and| energetic| personality|.

|One| day|,| Eugene| decided| to| embark| on| a| grand| adventure|.| He| set| off| on| a| journey| to| spread| happiness| and| positivity| throughout| the| kingdom|.| Along| the| way|,| he| encountered| people| from| all| walks| of| life| -| from| humble| farmers| to| noble| kni

Human (Q/q to quit):  q


AI: Bye bye human
